In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split  
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn import metrics  
from sklearn import tree
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

/Users/jkyung/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
df=pd.read_csv('billboard_hits_with_features.csv',index_col=0)

In [3]:
df.head()

,Year,Songs,Artists,Track_ID,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,2014,Happy,Pharrell Williams,spotify:track:6NPVjNh8Jhru9xOmyQigds,0.28600,0.652,233305,0.757,0.0,1,0.0886,-6.819,1,0.1530,159.911,4,0.962
1,2014,Dark Horse,Katy Perry featuring Juicy J,spotify:track:5jrdCoLpJSvHHorevXBATy,0.00314,0.645,215672,0.585,0.0,6,0.1650,-6.122,1,0.0513,131.931,4,0.353
2,2014,All of Me,John Legend,spotify:track:3U4isOIWM3VvDubwSI3y7a,0.92200,0.422,269560,0.264,0.0,8,0.1320,-7.064,1,0.0322,119.930,4,0.331
3,2014,Fancy,Iggy Azalea featuring Charli XCX,spotify:track:3oiMJQAWVaxSubJ7b2VUtX,0.10200,0.911,199938,0.707,0.0,8,0.0490,-4.136,1,0.0696,94.964,4,0.375
4,2014,Counting Stars,OneRepublic,spotify:track:6sy3LkhNFjJWlaeSMNwQ62,0.06490,0.663,257840,0.714,0.0,1,0.1160,-4.944,0,0.0380,121.990,4,0.468


In [4]:
df['top_10'] = 0
df['top_10'].loc[0:9] = 1
df['top_10'].loc[100:109] = 1
df['top_10'].loc[200:209] = 1
df['top_10'].loc[300:309] = 1

/Users/jkyung/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
df_train = df.iloc[:300, :] # 2014-2016
df_test = df.iloc[300:,:] # 2017

In [6]:
df_test.head()

,Year,Songs,Artists,Track_ID,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,top_10
300,2017,Shape of You,Ed Sheeran,spotify:track:7qiZfU4dY1lWllzX7mPBI3,0.581000,0.825,233713,0.652,0.000000,1,0.0931,-3.183,0,0.0802,95.977,4,0.931,1
301,2017,Despacito,Luis Fonsi and Daddy Yankee featuring Justin B...,spotify:track:6rPO02ozF3bM7NnOV4h6s2,0.222000,0.630,228827,0.815,0.000000,2,0.0910,-4.265,1,0.1520,88.357,4,0.813,1
302,2017,That's What I Like,Bruno Mars,spotify:track:0KKkJNfGyhkQ5aFogxQAPU,0.013000,0.853,206693,0.560,0.000000,1,0.0944,-4.961,1,0.0406,134.066,4,0.860,1
303,2017,Humble,Kendrick Lamar,spotify:track:7KXjTSCq5nL1LoYtL7XAwS,0.000259,0.904,177000,0.611,0.000020,1,0.0976,-6.842,0,0.0888,150.020,4,0.400,1
304,2017,Something Just Like This,The Chainsmokers and Coldplay,spotify:track:1dNIEtp7AY3oDAKCGg2XkH,0.030600,0.607,247627,0.649,0.000025,11,0.1740,-6.695,0,0.0362,102.996,4,0.505,1


In [7]:
print('Number of observations in the training data:', len(df_train))
print('Number of observations in the test data:',len(df_test))

Number of observations in the training data: 300
Number of observations in the test data: 100


In [8]:
df_features = df.drop(['Year', 'Songs','Artists','Track_ID','top_10'], axis=1)
features = df_features.columns[:]
features

Index(['acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

In [9]:
df_means = np.mean(df_train[features])
df_std = np.std(df_train[features])
df_train_features = (df_train[features] - df_means)/df_std
df_test_features = (df_test[features] - df_means)/df_std

In [10]:
y = df_train['top_10']

In [11]:
# Create a random forest Classifier. 
clf = RandomForestClassifier(n_jobs=2, random_state=0, n_estimators=1001, class_weight='balanced')

# Train the Classifier to take the training features and learn how they relate
# to the training y (the species)
clf.fit(df_train_features, y)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1001, n_jobs=2, oob_score=False, random_state=0,
            verbose=0, warm_start=False)

In [12]:
preds = clf.predict(df_test_features)
preds

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
clf.predict_proba(df_test_features)[0:10]

array([[ 0.8415738 ,  0.1584262 ],
       [ 0.89735528,  0.10264472],
       [ 0.912978  ,  0.087022  ],
       [ 0.87299454,  0.12700546],
       [ 0.95564436,  0.04435564],
       [ 0.90729271,  0.09270729],
       [ 0.31383879,  0.68616121],
       [ 0.78865685,  0.21134315],
       [ 0.94497515,  0.05502485],
       [ 0.93506494,  0.06493506]])

In [14]:
pd.crosstab(df_test['top_10'], preds, rownames=['Actual Hit'], colnames=['Predicted Hit'])

Predicted Hit,0,1
Actual Hit,,
0,89,1
1,9,1


In [15]:
list(zip(df_train[features], clf.feature_importances_))

[('acousticness', 0.087207032792433276),
 ('danceability', 0.098921327097575493),
 ('duration_ms', 0.096781889837850563),
 ('energy', 0.090343528195510836),
 ('instrumentalness', 0.051386637300595117),
 ('key', 0.073958175239767596),
 ('liveness', 0.085725469617879427),
 ('loudness', 0.10469079172206663),
 ('mode', 0.023683742577832505),
 ('speechiness', 0.087471801537929356),
 ('tempo', 0.091638094754767571),
 ('time_signature', 0.0026525647082364426),
 ('valence', 0.1055389446175555)]

In [16]:
# Logistic Regression
logreg = LogisticRegression()

In [17]:
logit_model=sm.Logit(y,df_train[features])
result=logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.302326
         Iterations 11
                           Logit Regression Results                           
Dep. Variable:                 top_10   No. Observations:                  300
Model:                          Logit   Df Residuals:                      287
Method:                           MLE   Df Model:                           12
Date:                Tue, 24 Jul 2018   Pseudo R-squ.:                 0.07000
Time:                        16:54:31   Log-Likelihood:                -90.698
converged:                       True   LL-Null:                       -97.525
                                        LLR p-value:                    0.3233
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
acousticness         0.4689      1.086      0.432      0.666      -1.660       2.598
danceabili